In [78]:
import sys
import numpy as np 
import os
import time as time_1
import math

from datetime import datetime
from pynq import Overlay
import pynq
import struct
from multiprocessing import Process, Pipe, Queue, Event, Manager

In [79]:
client_num = 6 # You can freely change the number of clients between 1-10 for this small scale design (float point)

In [ ]:
overlay = pynq.Overlay('GP_kernel.bit')
arraySize = 100
mu_buffer = pynq.allocate((100), np.float32)
a_buffer = pynq.allocate((100), np.float32)
beta_buffer = pynq.allocate((100), np.float32)
sigma_buffer = pynq.allocate((100, 100), np.float32)

mu_float = np.loadtxt("mu_float.txt", dtype = np.float32)
a_float = np.loadtxt("a_float.txt", dtype = np.float32)
beta_float = np.loadtxt("beta_float.txt", dtype = np.float32)
sigma_float = np.loadtxt("sigma_float.txt", dtype = np.float32)

np.copyto(mu_buffer, mu_float)
np.copyto(a_buffer, a_float)
np.copyto(beta_buffer, beta_float)
np.copyto(sigma_buffer, sigma_float)

client_number_buffer = pynq.allocate((10), np.int32)
client_index_list_buffer = pynq.allocate((10), np.int32)
result_buffer = pynq.allocate((100, 100), np.float32)
result_buffer_2 = pynq.allocate((100), np.float32)

client_number_buffer[0] = client_num
GP_kernel = overlay.GP_kernel_0

GP_kernel.write(0x10, mu_buffer.physical_address)
GP_kernel.write(0x1c, sigma_buffer.physical_address)
GP_kernel.write(0x28, a_buffer.physical_address)
GP_kernel.write(0x34, beta_buffer.physical_address)
GP_kernel.write(0x40, client_number_buffer.physical_address)
GP_kernel.write(0x4c, client_index_list_buffer.physical_address)
GP_kernel.write(0x58, result_buffer.physical_address)
GP_kernel.write(0x64, result_buffer_2.physical_address)

GP_kernel.write(0x00, 1)
isready = GP_kernel.read(0x00)

while( isready == 1 ):
    isready = GP_kernel.read(0x00)
    
print(client_index_list_buffer)
